# MarsToday: evaluating Mars climate through REMS sensor onboard Curiosity Mars rover

## Import box

In [1]:
#Imports
import pandas as pd
import numpy as np
import requests
from dotenv import load_dotenv
import os
from pandas import json_normalize
import re

## Data first visualization

In [2]:
weather = pd.read_csv("../data/mars-weather.csv")
weather

,id,terrestrial_date,sol,ls,month,min_temp,max_temp,pressure,wind_speed,atmo_opacity
0,1895,2018-02-27,1977,135,Month 5,-77.0,-10.0,727.0,NaN,Sunny
1,1893,2018-02-26,1976,135,Month 5,-77.0,-10.0,728.0,NaN,Sunny
2,1894,2018-02-25,1975,134,Month 5,-76.0,-16.0,729.0,NaN,Sunny
3,1892,2018-02-24,1974,134,Month 5,-77.0,-13.0,729.0,NaN,Sunny
4,1889,2018-02-23,1973,133,Month 5,-78.0,-18.0,730.0,NaN,Sunny
...,...,...,...,...,...,...,...,...,...,...
1889,24,2012-08-18,12,156,Month 6,-76.0,-18.0,741.0,NaN,Sunny
1890,13,2012-08-17,11,156,Month 6,-76.0,-11.0,740.0,NaN,Sunny
1891,2,2012-08-16,10,155,Month 6,-75.0,-16.0,739.0,NaN,Sunny
1892,232,2012-08-15,9,155,Month 6,NaN,NaN,NaN,NaN,Sunny


## Connection with NASA API

In [18]:
load_dotenv()

True

In [23]:
# This is a function to call NASA api with a given value of Sol day and Camera type.

def call_NASA (sol,camera):
    """
    This is a function that calls NASA API 'Mars Rover Photos' with two arguments. It returns the url from
    a specific camera onboard Curiosity rover.
    sol: input Sol value (1-1894) as INTEGER,
    camera: select between FHAZ, RHAZ, MAST, CHEMCAM, MAHLI, MARDI, NAVCAM, PANCAM, MINITES, as STRING
    
    """
        
    try:
        nasa = os.getenv("token")
        url = f"https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos?sol={sol}&camera={camera}&api_key={nasa}"
        request = requests.get(url)
        df = pd.DataFrame(request.json())
        df_clean = pd.DataFrame(df.values[0][0])
        image_url = list(df_clean["img_src"])[0]
                
        return f"Image available for camera {camera}: {image_url}"
            
    except:
            
            return f"No image available in Sol {sol} for camera {camera}, please select another Sol day"    

In [24]:
def get_pictures(sol):
    
    """
    This is a function that calls call_NASA function with one argument. It returns the url of all the pictures 
    taken by all the cameras of Curiosity rover from a specific Sol date.
    sol: input Sol value (1-current sol number) as INTEGER.
    
    """       
    cameralist = ["FHAZ", "RHAZ", "MAST", "CHEMCAM", "MAHLI", "MARDI", "NAVCAM", "PANCAM", "MINITES"]
    for i in cameralist:
        print(call_NASA(sol, i))
    pass

# API call

In [31]:
get_pictures(3626)

Image available for camera FHAZ: https://mars.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/03626/opgs/edr/fcam/FLB_719396719EDR_F0971918FHAZ00302M_.JPG
Image available for camera RHAZ: https://mars.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/03626/opgs/edr/rcam/RLB_719396766EDR_S0971918RHAZ00300M_.JPG
Image available for camera MAST: https://mars.nasa.gov/msl-raw-images/msss/03626/mcam/3626ML1029340040209331C00_DXXX.jpg
Image available for camera CHEMCAM: https://mars.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/03626/opgs/edr/ccam/CR0_719388758EDR_F0971734CCAM02626M_.JPG
Image available for camera MAHLI: https://mars.nasa.gov/msl-raw-images/msss/03626/mhli/3626MH0001220011301172C00_DXXX.jpg
Image available for camera MARDI: https://mars.nasa.gov/msl-raw-images/msss/03626/mrdi/3626MD0013580000300001E01_DXXX.jpg
Image available for camera NAVCAM: https://mars.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/03626/opgs/edr/ncam/NLB_719400712EDR_F0